In [194]:
inputFile=r"C:\Users\jdub\Downloads\Lab6\Lab6\data.xlsx"
import gurobipy as grb
import pandas as pd
data=pd.read_excel(inputFile,sheetname=None,index_col=0)
prefs=data['Preferences']
shifts=data['Shifts']

In [195]:
I = prefs.index #nurses
J = shifts.index #shifts

mod=grb.Model()
#Descision vars
X={}
for i in I:
    for j in J:
        X[i,j]=mod.addVar(lb=0, ub=1, vtype=grb.GRB.BINARY, name='x[{0},{1}]'.format(i,j))

In [196]:
numShifts = len(J) #shift count
D = range(int(numShifts/3)) #days

#Non consecutive contraint
for i in I:
    for j in J:
        if j+1 < numShifts:
            mod.addConstr(X[i,j]+X[i,j+1] <= 1)
            
#Night constraint
for i in I:
    for d in D:
        j = 3*d+2
        mod.addConstr(X[i,j]+X[i,j-2] <= 1)
        if j+2 < numShifts:
            mod.addConstr(X[i,j]+X[i,j+2] <= 1)

#Shift quota constraint
for j in J:
    mod.addConstr(sum((X[i,j] for i in I)) == shifts.loc[j,'persons'])
    
#6 shift maximum constraint
W = range(int(numShifts/21)) #weeks
for i in I:
    for w in W:
        shiftSet = range(w*21,(w+1)*21)
        mod.addConstr(sum((X[i,j] for j in shiftSet)) <= 6)

#Blackout constraint
for i in I:
    for j in J:
        if prefs.loc[i,j] == 0:
            mod.addConstr(X[i,j] == 0)

In [197]:
#Sum of preference scores
P = sum((prefs.loc[i,j]*X[i,j] for i in I for j in J))
    
# #Shift inequality
U=mod.addVar(lb=0, ub=numShifts, vtype=grb.GRB.INTEGER, name='Shift Inequality')
L=mod.addVar(lb=0, ub=numShifts, vtype=grb.GRB.INTEGER, name='Shift Inequality')
for i in I:
    mod.addConstr(sum(X[i,j] for j in J)<=U)
    mod.addConstr(sum(X[i,j] for j in J)>=L)

#Night inequality
U_N=mod.addVar(lb=0, ub=int(numShifts/3), vtype=grb.GRB.INTEGER, name='NIght Inequality')
L_N=mod.addVar(lb=0, ub=int(numShifts/3), vtype=grb.GRB.INTEGER, name='NIght Inequality')
for i in I:
    mod.addConstr(sum(X[i,3*d+2] for d in D)<=U_N)
    mod.addConstr(sum(X[i,3*d+2] for d in D)>=L_N)

mod.setObjective(P - 100*(U-L) - 100*(U_N-L_N) , sense=grb.GRB.MAXIMIZE)
# mod.setObjective(P , sense=grb.GRB.MAXIMIZE)

In [198]:
mod.setParam('OutputFlag',False)   
mod.optimize()
print('Optimal objective: {0:.2f}'.format(mod.ObjVal))

Optimal objective: 4117.00


In [199]:
solutions = []
for i,x in enumerate(X):
    solutions.append((x[0], x[1], int(mod.getVars()[i].x)))
pd.DataFrame(solutions, columns = ['Name','Shift','Value']).pivot_table(index='Name',columns='Shift')

Value                                     ...                      \
Shift         0   1   2   3   4   5   6   7   8   9   ... 179 180 181 182 183   
Name                                                  ...                       
Alexis          0   0   0   0   1   0   1   0   0   0 ...   1   0   0   0   0   
Alyssa          1   0   0   0   1   0   0   0   1   0 ...   1   0   0   1   0   
Anthony         0   0   0   1   0   0   1   0   0   1 ...   0   1   0   0   0   
Brandon         0   0   0   0   1   0   1   0   0   1 ...   0   0   1   0   0   
Brianna         1   0   0   1   0   0   0   1   0   0 ...   0   0   0   0   0   
Caleb           0   0   0   1   0   0   0   1   0   1 ...   0   1   0   0   0   
Cameron         0   1   0   0   0   1   0   0   0   0 ...   0   0   1   0   0   
Chloe           0   0   1   0   0   0   1   0   0   0 ...   0   0   0   0   0   
Christopher     0   1   0   1   0   0   0   1   0   0 ...   0   0   0   0   1   
Daniel          1   0   0   1   0   0   0   1   0   0 ...   0   1   0   0   0   
David           0   1   0   0   1   0   0   0   0   1 ...   0   1   0   0   0   
Destiny         0   1   0   0   1   0   1   0   0   0 ...   0   1   0   0   1   
Dorothy         0   0   1   0   0   1   0   0   0   1 ...   0   0   1   0   0   
Ethan           1   0   0   0   0   0   0   1   0   0 ...   0   0   0   0   0   
Gabriel         1   0   0   0   1   0   0   0   0   0 ...   0   0   1   0   0   
Hailey          0   1   0   0   0   0   0   1   0   1 ...   0   0   1   0   1   
Hannah          0   1   0   0   0   0   1   0   0   0 ...   0   0   0   1   0   
Imani           0   1   0   1   0   0   0   0   1   0 ...   0   1   0   0   0   
Isaac           0   1   0   1   0   0   1   0   0   0 ...   0   1   0   0   1   
Isaiah          1   0   0   0   1   0   0   0   1   0 ...   0   1   0   0   0   
James           0   1   0   0   1   0   0   0   0   0 ...   1   0   0   0   0   
Jasmine         1   0   0   0   1   0   0   1   0   0 ...   0   0   0   0   1   
Jayden          1   0   0   0   0   0   1   0   0   0 ...   0   1   0   0   0   
Jayla           0   0   0   0   1   0   1   0   0   0 ...   0   1   0   0   1   
Jeremiah        0   0   0   0   0   0   0   1   0   1 ...   0   0   1   0   0   
Jordan          1   0   0   1   0   0   0   0   1   0 ...   0   1   0   0   0   
Joseph          1   0   0   1   0   0   0   0   0   0 ...   0   0   0   1   0   
Joshua          1   0   0   0   1   0   0   1   0   0 ...   0   0   0   0   1   
Josiah          0   1   0   1   0   0   0   0   0   1 ...   0   1   0   0   0   
Justin          0   1   0   0   0   0   1   0   0   0 ...   0   0   1   0   0   
Kayla           0   0   0   1   0   0   0   0   0   0 ...   0   1   0   0   0   
Kennedy         0   0   1   0   0   1   0   0   1   0 ...   0   0   0   0   1   
Kevin           1   0   0   0   0   0   0   0   0   1 ...   0   0   0   0   0   
Kiara           0   0   1   0   0   0   1   0   0   0 ...   1   0   0   0   1   
Madison         0   1   0   0   1   0   0   0   0   1 ...   1   0   0   0   0   
Makayla         0   0   1   0   0   1   0   0   0   0 ...   0   1   0   0   0   
Malik           1   0   0   1   0   0   0   1   0   0 ...   0   0   1   0   0   
Matthew         0   1   0   1   0   0   1   0   0   1 ...   0   0   0   0   0   
Michael         0   0   0   0   1   0   1   0   0   0 ...   0   0   0   0   0   
Nathan          0   0   0   0   0   0   0   1   0   1 ...   0   0   1   0   1   
Nevaeh          0   0   0   0   0   0   0   1   0   0 ...   0   0   1   0   1   
Patricia        0   0   0   0   0   0   1   0   0   0 ...   0   0   0   1   0   
Susan           1   0   0   0   0   0   0   1   0   0 ...   0   0   1   0   1   
Sydney          0   0   0   1   0   0   0   1   0   0 ...   0   0   1   0   0   
Taylor          0   0   0   0   0   0   1   0   0   1 ...   0   0   1   0   1   
Tiana           0   0   0   1   0   0   0   0   0   1 ...   0   1   0   0   1   
Trinity         0   1   0   0   0   0   0  